# NoSQL - MongoDB 2 Exercise

## MongoDB Recap

### Question 1
Use [JSONlint](https://jsonlint.com/) to check your work.

The questions will be using the following `Movie` JSON document:
```json
{
  "id" : 14253,
  "title" : "Beauty and the Beast",
  "year" : 2016,
  "language" : "English",
  "imdb_rating" : 6.4,
  "genre" : "Romance",
  "director" : "Christophe Gans",
  "runtime" : 112
}
```

**a)** Add the a document as an embedded document to the `imdb_rating` field. This document has the fields `rating` & `votes` (fill in with whatever values you like for those fields).

**b)** We are now going to create a `tomatometer` rating that includes a `viewer_ratings` and `critics_ratings` document. Each of those ratings will have a `rating` field and a `votes` field (fill in with whatever values you like for those fields). In addition, the `Tomatometer` rating also as the `fresh` & `rotten` fields (fill in with whatever values you like for those fields). The `tomatometer` rating is to be added to the `Movie` JSON document.

**c)** We are going to add a document call `comments` to the `Movie` JSON document. Create 2 comment documents with the fields `name`, `email`, `text` and `date` (fill in with whatever values you like for those fields). These comments are to be added as an array to the `Movie` JSON document

### Answer Question 1
Your final `Movie` JSON document should be similar to this
```json
{
    "id" : 14253,
    "title" : "Beauty and the Beast",
    "year" : 2016,
    "language" : "English",
    "imdb_rating" : { "rating":6.4, "votes": 17762 }
    "genre" : "Romance",
    "director" : "Christophe Gans",
    "runtime" : 112,
    "tomatometer" :
    {
        "viewer_ratings": { "rating": 3.9, "votes": 258 },
        "critics_ratings": { "rating": 4.9, "votes": 50 },
        "fresh": 96,
        "rotten": 7
    },
    "comments" : 
    [ 
        {
            "name" : "Tom Goodwin",
            "email" : "tom.goodwin@newamsterdam.com"
            "text" : "Rem itaque ad sit rem voluptatibus. Ad fugiat...",
            "date" : "2017-08-22T11:45:03.000+00:00"
        },
        {
            "name" : "Melly Butcher",
            "email" : "m.butcher@gameofthrones.com"
            "text" : "Perspiciatis non debitis magnam. Voluptate...",
            "date" : "2018-08-22T11:45:03.000+00:00"
        }
    ]
}
```

### Question 2

This question will be using the MongoDB collection called `restaurants` from the previous MongoDB exercise.

The full collection records are in the file called `restaurants.json`. If you would like to import the collection into MongoDB, use the following command in the command prompt window.

`mongoimport --db=<database name> --collection=<collection name> --file=<full json file path & name>`

<br>

Using the given information, answer the following questions:

1. Write a query to find the restaurants which belong to the borough Bronx and prepared either American or Chinese dish. Show only the `name`, `address` & `grades` of the documents.

2. Write a query to find the restaurants name, address, borough and cuisine for those restaurants which achieved a score which is not more than 10.

3. Write a query to add a `comments` field to all restaurants. Leave the a `NULL` value for the comment field.

### Answer Question 2
```
1. db.resturants.find({"borough": "Bronx", $or : [{"cuisine": "American"}, {"cuisine": "Chinese"}]}, {_id:0, name:1, address:1,grades:1})

2. db.resturants.find({"grades.score": {$not : {$gt : 10}}}, {_id:0, name:1, address:1,borough:1, cuisine:1})

3. db.resturants.updateMany({}, {$set: {"comments": null}})
```

---
## MongoDB Aggregation Framework

In this section, we will be using the MongoDB sample **movie** database. If you have already setup MongoDB Atlas, feel free to use it (the Movies dataset is part of the sample dataset provided for MongoDB Altas) otherwise the sample database can be downloaded from Moodle and loaded onto your local MongoDB installation via `mongorestore` command. Since we will be loading the **whole database from the folder (not just a collection)**, we have to use the following command.

`mongorestore --db=<database name> <full directory path>`

Within the movies dataset, we have 5 collections:

| Collection Name | Description |
|:---|:---|
| comments | Contains comments associated with specific movies. |
| movies | Contains movie information, including release year, director, and reviews. |
| sessions | Metadata field. Contains users’ JSON Web Tokens. |
| theaters | Contains locations of movie theaters. |
| users | Contains user information.|

We will not be using all the collections but it is advisable to play around with the dataset.

Once you have completed any question, you can try to output the results to another collection to practice using the `$out` or `$merge` stages.

### Question 1
Show the top three movies in the romance genre sorted by IMDb rating (highest to lowest), and only movies released before 2001. Using the fields `released` or `year` is fine. <br>
 **Output:**
 ```json
{
        "genres" : [
                "Drama",
                "Romance"
        ],
        "title" : "Pride and Prejudice",
        "released" : ISODate("1996-01-14T00:00:00Z"),
        "year" : 1995,
        "imdb" : {
                "rating" : 9.1
        }
}
{
        "imdb" : {
                "rating" : 8.8
        },
        "year" : 1994,
        "genres" : [
                "Drama",
                "Romance"
        ],
        "title" : "Forrest Gump",
        "released" : ISODate("1994-07-06T00:00:00Z")
}
{
        "genres" : [
                "Comedy",
                "Family",
                "Romance"
        ],
        "title" : "Andaz Apna Apna",
        "released" : ISODate("1994-11-04T00:00:00Z"),
        "year" : 1994,
        "imdb" : {
                "rating" : 8.8
        }
}
 ```
 

### Answer Question 1

```
db.movies.aggregate( [
  { $match: {year: {$lt: 2001}, genres: "Romance" } },
  { $sort: {"imdb.rating": -1}},
  { $limit: 3},
  { $project: {_id: 0, title: 1, year:1, released: 1, "imdb.rating": 1, genres: 1} }
]).pretty()
```

### Question 2

Find the top 3 Award-Winning Documentary movies.

 **Output**<br>
 ```json
{
        "genres" : [
                "Documentary",
                "History"
        ],
        "title" : "The Act of Killing",
        "awards" : {
                "wins" : 57,
                "nominations" : 30,
                "text" : "Nominated for 1 Oscar. Another 56 wins & 30 nominations."
        }
}
{
        "genres" : [
                "Documentary"
        ],
        "title" : "Citizenfour",
        "awards" : {
                "wins" : 44,
                "nominations" : 25,
                "text" : "Won 1 Oscar. Another 43 wins & 25 nominations."
        }
}
{
        "genres" : [
                "Documentary",
                "Biography",
                "Music"
        ],
        "title" : "Searching for Sugar Man",
        "awards" : {
                "wins" : 43,
                "nominations" : 26,
                "text" : "Won 1 Oscar. Another 42 wins & 26 nominations."
        }
}
 ```
 

### Answer Question 2

```
db.movies.aggregate( [
  { $match: { "awards.wins": {$gte: 1}, genres: "Documentary" } },
  { $sort: {"awards.wins": -1} },
  { $limit: 3},
  { $project: { _id: 0, title: 1, genres: 1, awards: 1 } }
]).pretty()
```

### Question 3

Find the average runtime of all movies by ratings, truncate to the nearest 2 decimal places. You will need the `$trunc` aggregation operator. You may also want to read up on the `$round` aggregation operator and understand why it may not be a good operator to use in many cases.
 
 **Small sample output**<br>
 ```json
{ "_id" : "G", "roundedAvgRuntime" : 90.81 }
{ "_id" : "APPROVED", "roundedAvgRuntime" : 105.17 }
{ "_id" : "TV-G", "roundedAvgRuntime" : 80.22 }
 ```

### Answer Question 3

```
db.movies.aggregate( [
  { $group: {_id: "$rated", "avgRunTime": {$avg: "$runtime"} } },
  { $project: {"roundedAvgRuntime": {$trunc: ["$avgRunTime", 2]} } }
]).pretty()
```

### Question 4

For only movies older than 2001, find the average and maximum popularity for each genre, sort the genres by popularity, and find the adjusted (with trailers) runtime of the longest movie in each genre. Assume that the first genre in the genre array is the primary genre of a film and movies trailers always runs for 12 mins. Using the fields `released` or `year` is fine. <br>

 If your results are correct, you should get 23 records and the top 3 documents are
 ```json
{
        "_id" : "Film-Noir",
        "popularity" : 7.62,
        "max_popularity" : 8.3,
        "adjusted_runtime" : 123
}
{
        "_id" : "Documentary",
        "popularity" : 7.5219954648526075,
        "max_popularity" : 9.4,
        "adjusted_runtime" : 1152
}
{
        "_id" : "Short",
        "popularity" : 7.313235294117647,
        "max_popularity" : 8.6,
        "adjusted_runtime" : 56
}
 ```

### Answer Question 4

```
db.movies.aggregate( [
  { $match: {year: {$lt: 2001} } },
  { $group: {
      _id: {"$arrayElemAt": ["$genres", 0]},
      "popularity": {$avg: "$imdb.rating"},
      "max_popularity": {$max: "$imdb.rating"},
      "longest_runtime": {$max: "$runtime"}
  } },
  { $sort: {popularity: -1}},
  { $project: {
      popularity: 1, max_popularity:1, 
      adjusted_runtime: {$add: ["$longest_runtime", 12]} 
  } }
]).pretty()
```

### Question 5

Continuation of Question 4. Select a title from each category based on the following requirements:
* the cinema informed you that they can only show movies less than or equal to 230 mins long (aka 218 mins movie length + 12 mins trailer).
* the movie imdb rating have to be greater than or equal to 7.0
* you will need to find out how to use the `$first` aggregation operator
 
If your results are correct, you should get 23 records (same as previous) and the top 2 documents are

```json
{
        "_id" : "Film-Noir",
        "recommended_title" : "The Third Man",
        "recommended_rating" : 8.3,
        "recommended_raw_runtime" : 93,
        "popularity" : 7.85,
        "max_popularity" : 8.3,
        "adjusted_runtime" : 123
}
{
        "_id" : "Documentary",
        "recommended_title" : "Cosmos",
        "recommended_rating" : 9.3,
        "recommended_raw_runtime" : 60,
        "popularity" : 7.692458100558659,
        "max_popularity" : 9.3,
        "adjusted_runtime" : 212
}
 ```
 

### Answer Question 5

```
db.movies.aggregate( [
  { $match: {year: {$lt: 2001}, runtime: {$lte: 218}, "imdb.rating":{$gte: 7.0} } },
  { $sort: {"imdb.rating": -1}},
  { $group: {
      _id: {"$arrayElemAt": ["$genres", 0]},
      "recommended_title": {$first: "$title"},
      "recommended_rating": {$first: "$imdb.rating"},
      "recommended_raw_runtime": {$first: "$runtime"},
      "popularity": {$avg: "$imdb.rating"},
      "max_popularity": {$max: "$imdb.rating"},
      "longest_runtime": {$max: "$runtime"}
  } },
  { $sort: {popularity: -1}},
  { $project: {
      popularity: 1, max_popularity:1, recommended_title: 1,
      recommended_rating: 1, recommended_raw_runtime: 1, 
      adjusted_runtime: {$add: ["$longest_runtime", 12]} 
  } }
]).pretty()
```

### Question 6

For this question, we will work with the stage `$sample`. This stage allow us to use a smaller dataset especially when we are working with huge datasets. The syntax for this stage is `{ $sample: { size: <positive integer> } }`. 

**Note that `$sample` stage will select at random the documents used for the sample size.** 

Let's see the output of the code that searches for the word "around" in the `plot` field: <br>

```
db.movies.aggregate( [
    { $match: {plot: {$regex: /around/} } }
])
```
 
from the output, we can tell that it is A LOT (371 documents, in fact). Let's define a sample size <br>

```
db.movies.aggregate( [
    { $sample: {size: 20} },
    { $match: {plot: {$regex: /around/} } }
])
```
 
The numbers of documents have greatly reduced and the search time is now faster. You can run this aggregation using Pymongo and compare the search times.

### Question 7

For this question, we will work with the stage `$lookup` that allows us to write queries across multiple collections. <br>
The general syntax for the `$lookup` stage is:
 
```
{
   $lookup:
     {
       from: <collection to join>,
       localField: <field from the input documents>,
       foreignField: <field from the documents of the "from" collection>,
       as: <output array field>
     }
}
```
 
We will be joining data from the collections `users` and `comments`. We want to find all comments made by the users `Catelyn Stark` and `Ned Stark`. The way to use the `$lookup` stage is to decide the relationship that links the 2 collections (similar to RDBMS primary & foreign keys).
  
The 4 parameters are as follows:
* `from` - the collection we are joining to our current aggregation. In this case, we are joining `comments` to `users`.
* `localField` - the field name that we are going to use to join our documents in the local collection (ie. the collection we are running the aggregation on). In this case, the name of our user.
* `foreignField` - the field that links to `localField` in the `from` collection. These may have different names, but in this scenario, it is the same field `name`.
* `as` - this is how our new joined data will be labeled.
  
<br>
  
Create this aggregation pipeline and **limit your output to 2 documents**. Because of the large number of comments, we may want to unwind the array and just show the first 2 comments.

### Answer Question 7

```
db.users.aggregate( [
  { $match: { $or: [{name: "Catelyn Stark"}, {name: "Ned Stark"}] } },
  { $lookup: {
      from: "comments", 
      localField: "name",
      foreignField: "name",
      as: "comments"
  } },
  { $unwind: "$comments"},
  { $limit: 2}
])
```

### Question 8

Show or output to another collection a list of movies that had generated the most comments from users. <br>
**Hint**, You will need the following stages template and the following steps:
 
```
db.comments.aggregate( [
    { $sample: {}},
    { $group: {}},
    { $sort: {}},
    { $limit: 5},
    { $lookup: {}},
    { $unwind: },
    { $project: {}},
    { $merge: {}}
])
```
 
1. Check how large the `comments` collection is before deciding on a sample size, use the `count()` function. Sample size should be about 10% of population size.
2. Group the comments by their associated film, accumulating the total number of comments for each film. You'll need to find out how to sum all documents for each group.
3. Sort via most comments (aka result from step 2). You may want to run the aggregate function at this point to check your progress.
4. Link to the `movies` collection using the `_id` field.
5. Unwind the results from step 4.
6. Show only the movie's title, imdb ratings and the total number of comments.
7. Optional. Save the results to another collection.
 
 
**Output without the sample stage**

```json
{ "_id" : ObjectId("573a13bff29313caabd5e91e"), "sumComments" : 161, "movie" : { "imdb" : { "rating" : 6.4 }, "title" : "The Taking of Pelham 1 2 3" } }
{ "_id" : ObjectId("573a13a5f29313caabd159a9"), "sumComments" : 158, "movie" : { "imdb" : { "rating" : 7.1 }, "title" : "About a Boy" } }
{ "_id" : ObjectId("573a13abf29313caabd25582"), "sumComments" : 158, "movie" : { "imdb" : { "rating" : 6.8 }, "title" : "50 First Dates" } }
{ "_id" : ObjectId("573a13b3f29313caabd3b647"), "sumComments" : 158, "movie" : { "imdb" : { "rating" : 6.7 }, "title" : "Terminator Salvation" } }
{ "_id" : ObjectId("573a13a3f29313caabd0d1e3"), "sumComments" : 158, "movie" : { "imdb" : { "rating" : 7.8 }, "title" : "Ocean's Eleven" } }
```

### Answer Question 8

```
db.comments.aggregate( [
    { $sample: { size: 5000}},
    { $group: { 
        _id: "$movie_id",
        "sumComments": {$sum: 1}
    }},
    { $sort: {"sumComments": -1}},
    { $limit: 5},
    { $lookup: {
        from: "movies",
        localField: "_id",
        foreignField: "_id",
        as: "movie"
    }},
    { $unwind: "$movie"},
    { $project: {
        "movie.title": 1,
        "movie.imdb.rating": 1,
        "sumComments": 1
    }}
    { $merge: "most_commented_movies"}
])
```